In [1]:
from gevent import monkey
monkey.patch_all()
from gevent.queue import Queue
import gevent

import sys
sys.path.append('/Users/hzm/Desktop/YangTeng/code/venv/code/00.module')
import Proxy
import UA
import time

# = = = = = = = = = = = = = = = = = =

import pandas as pd
from selenium.webdriver import ChromeOptions

option = ChromeOptions()
option.add_experimental_option('useAutomationExtension', False)
option.add_argument('blink-settings=imagesEnabled=false')
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument('--disable-dev-shm-usage')
option.add_argument('--disable-gpu')
option.add_experimental_option('excludeSwitches', ['enable-automation'])
option.add_argument('--headless')
option.add_argument('--no-sandbox')
option.add_argument('--proxy-server=' + Proxy.get_Proxy_selenium())
option.add_argument('user-agent=' + UA.get_UA())
option.page_load_strategy = 'eager'

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

# = = = = = = = = = = = = = = = = = =

df_menu = pd.read_excel('./menu.xlsx', header=0)
list_url = df_menu['Url'].to_list()

print('总数量：' + str(len(list_url)))
print()

df = pd.DataFrame()

# = = = = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(11):
            try:
                browser = Chrome(service=Service('../../../chromedriver'), options=option)
                browser.maximize_window()

                # = = = = = = = = = = = = = = = = = =

                browser.get(url)
                
                time.sleep(3)
                
                # = = = = = = = = = = = = = = = = = =
                
                if len(browser.find_elements(by=By.XPATH, value='//span[contains(@class, "navlabellink") and contains(@class, "nvoffset") and contains(@class, "nnormal")]')) != 0:
                    break
                    
                # = = = = = = = = = = = = = = = = = =

                wait = WebDriverWait(browser, 60)
                wait.until(EC.presence_of_element_located((By.XPATH, '//tbody[contains(@class, "listing-inner") and contains(@class, "altrow-a-1")]')))
                list_part = browser.find_elements(by=By.XPATH, value='//tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-a-1") or contains(@class, "altrow-a-0"))]')

                # = = = = = = = = = = = = = = = = = =

                list_part_number = [part.find_element(by=By.XPATH, value='./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@title, "Buyer\'s Guide")]') for part in list_part]

                # = = = = = = = = = = = = = = = = = =
                
                part_index = [part_number.text for part_number in list_part_number].index(url.split(',')[-3])
                
                # = = = = = = = = = = = = = = = = = =
                
                manufacturer = list_part[part_index].find_element(by=By.XPATH, value='./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@class, "listing-final-manufacturer")]').text

                # = = = = = = = = = = = = = = = = = =
                
                list_part_number[part_index].click()
                time.sleep(7)
                list_vehicle = browser.find_element(by=By.XPATH, value='//div[contains(@id, "buyersguidepopup-outer_c")]').find_elements(by=By.XPATH, value='./descendant::table[contains(@class, "nobmp")]')
                if len(list_vehicle) == 0:
                    vehicle = ''
                else:
                    vehicle = list_vehicle[0].text

                # = = = = = = = = = = = = = = = = = =

                note_1 = ';'.join(row.text for row in list_part[part_index].find_elements(by=By.XPATH, value='./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@class, "span-link-underline-remover")]'))

                # = = = = = = = = = = = = = = = = = =

                info = ';'.join(row.get_attribute('href') for row in list_part[part_index].find_elements(by=By.XPATH, value='./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/a[contains(@class, "ra-btn") and contains(@class, "ra-btn-moreinfo")]'))

                # = = = = = = = = = = = = = = = = = =
                
                note_2 = ';'.join(row.text for row in list_part[part_index].find_elements(by=By.XPATH, value='./descendant::div[contains(@class, "listing-text-row") and not(contains(@class, "moreinfo-truck"))]'))

                # = = = = = = = = = = = = = = = = = =

                oe = ';'.join(row.get_attribute('textContent') for row in list_part[part_index].find_elements(by=By.XPATH, value='./descendant::span[contains(@title, "Replaces these Alternate/ OE Part Numbers")]'))
                oe = ';'.join(oe.split(', '))
                
                # = = = = = = = = = = = = = = = = = =

                src = ';'.join(row.get_attribute('src') for row in list_part[part_index].find_elements(by=By.XPATH, value='./descendant::img[contains(@id, "inlineimg_thumb")]'))

                # = = = = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'status': 'ok',
                                         'Manufacturer': manufacturer,
                                         'Vehicle': vehicle,
                                         'Alternate_OE_Part_Numbers': oe,
                                         'Note_1': note_1,
                                         'Note_2': note_2,
                                         'Pic': '',
                                         'Url': url,
                                         'Info': info,
                                         'Src': src}])
                
                browser.quit()
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(3)
                continue
                
        # = = = = = = = = = = = = = = = = = =

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Url': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url.split(',')[-3] +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))
        
# = = = = = = = = = = = = = = = = = =

list_task = []
for _ in range(25):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('输出中...')
df.to_excel('./part.xlsx', index=False)
print()
print('搞定！')

总数量：437

31045SA  <->  [ok] - 剩余数量：412
31036SA  <->  [ok] - 剩余数量：411
31041S  <->  [ok] - 剩余数量：410
30250S  <->  [ok] - 剩余数量：409
31007S  <->  [ok] - 剩余数量：408
31047S  <->  [ok] - 剩余数量：407
30719S  <->  [ok] - 剩余数量：406
30470S  <->  [ok] - 剩余数量：405
30490SX  <->  [ok] - 剩余数量：404
30460SA  <->  [ok] - 剩余数量：403
30470SX  <->  [ok] - 剩余数量：402
31035S  <->  [ok] - 剩余数量：401
31038S  <->  [ok] - 剩余数量：400
30460SB  <->  [ok] - 剩余数量：399
31045S  <->  [ok] - 剩余数量：398
31035SX  <->  [ok] - 剩余数量：397
30460S  <->  [ok] - 剩余数量：396
30430S  <->  [ok] - 剩余数量：395
31036S  <->  [ok] - 剩余数量：394
30420S  <->  [ok] - 剩余数量：393
31047SHWP  <->  [ok] - 剩余数量：392
31047SH  <->  [ok] - 剩余数量：391
30250SX  <->  [ok] - 剩余数量：390
30719SHWP  <->  [ok] - 剩余数量：389
31038SX  <->  [ok] - 剩余数量：388
31049SAH  <->  [ok] - 剩余数量：387
31049SAHWP  <->  [ok] - 剩余数量：386
31047SXHWP  <->  [ok] - 剩余数量：385
31047SXH  <->  [ok] - 剩余数量：384
31049SE  <->  [ok] - 剩余数量：383
31049SB  <->  [ok] - 剩余数量：382
31049SBXH  <->  [ok] - 剩余数量：381
31049SCH  <->  [ok] - 剩余数量：380

KeyboardInterrupt
2023-08-23T01:11:49Z


KeyboardInterrupt: 